<b>Description</b>

This data loader python scripts processes the item list by:

(1) Downloading the item's image through "imUrl" attribute

(2) Analyzing the image using rekognition and retrieving the labels. Confidence level threshold is 60%

(3) Load the data into DynamoDB

(4) Format the data to item.csv which can be consumed by Personalized as item category data. 

Import dependencies and libraries


In [2]:
import boto3
import json
import logging
import requests
import pandas as pd
import csv


Change "INPUT DynamoDB Table Name" to the name of the DynamoDB table you plan to write to.

Ensure that items.json file is in the local file system

In [ ]:
import boto3
import json
import logging
import requests
import pandas as pd


dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('ItemsTest')

with open('items.json') as json_file:
    items = json.load(json_file)
    with open("./item.csv", "w") as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            columnheader=[]
            columnheader.append("ITEM_ID")
            columnheader.append("GENRE")
            writer.writerow(columnheader)
            for x in items:
                asin = x['asin']
                if asin == "":
                    continue
                    
                title = x['title']
                if title == "":
                    continue
                    
                imUrl = x['imUrl']

                if imUrl == "":
                    continue
                
                try:
                    img = requests.get(imUrl).content
                    client = boto3.client('rekognition')
                    response = client.detect_labels(
                    Image={
                        "Bytes" : img
                    },
                    MaxLabels=123,
                    MinConfidence=60
                    )
                except :
                    continue


                for key, value in response.items():
                    if key == 'Labels':
                        labels = value

                df = pd.DataFrame(labels)
                if df.empty:
                    continue
                df2 = df

                for col in df2.columns: 
                    if col == "Parents":
                        df2=df2.drop(['Parents'], axis=1)
                    elif col == "Instances":
                        df2=df2.drop(['Instances'], axis=1)
                print ("******")
                print (df2)
                jsonOb = df2["Name"].to_json(orient='records')

                table.put_item(
                    Item={
                        'asin': asin,
                        'title': title,
                        'imUrl':imUrl, 
                        'rekognition': jsonOb

                    }
                )
                
                csvrow = []
                categories =""
                for x in df2["Name"]:
                    categories += x
                    categories += "|"
    
                csvrow.append(asin)
                csvrow.append (categories)
                writer.writerow(csvrow)


************
******
   Confidence        Name
0   99.831238        Soda
1   99.831238       Drink
2   99.831238    Beverage
3   94.991508     Ketchup
4   94.991508        Food
5   76.706963      Bottle
6   76.706963  Pop Bottle
7   70.760788        Coke
8   70.760788        Coca
************
******
   Confidence         Name
0   90.163925    Accessory
1   90.163925       Wallet
2   90.163925  Accessories
3   73.416534      Adapter
************
******
   Confidence          Name
0   97.263611        Camera
1   97.263611   Electronics
2   82.886307  Video Camera
3   60.485237    Binoculars
************
******
   Confidence           Name
0   91.494095  Digital Watch
************
******
   Confidence       Name
0   95.222954  Lamp Post
1   83.826889       Oars
2   61.786564     Symbol
************
******
   Confidence      Name
0   80.437836      Rope
1   67.307632  Clothing
2   67.307632   Apparel
************
******
   Confidence        Name
0   88.868462       Sport
1   88.868462      

************
******
   Confidence        Name
0   97.690552      Weapon
1   97.690552    Weaponry
2   93.401321  Ammunition
3   78.115356      Bullet
************
******
   Confidence        Name
0   97.562164      Weapon
1   97.562164    Weaponry
2   94.662827  Ammunition
3   81.230980      Bullet
************
******
   Confidence        Name
0   92.194183         Rug
1   89.090858       Towel
2   89.090858  Bath Towel
************
******
   Confidence    Name
0   79.045395    Tool
1   77.372643   Clamp
2   63.103390  Buckle
************
******
   Confidence   Name
0   99.695999   Tool
1   97.738121  Clamp
************
******
   Confidence         Name
0   99.967690    Accessory
1   99.967690   Sunglasses
2   99.967690  Accessories
3   99.466576      Glasses
4   83.921333      Goggles
************
******
   Confidence      Name
0   92.597496  Lighting
1   66.391853       Bow
2   63.401180   Machine
************
******
   Confidence           Name
0   94.830536           Tool
1   85.31

************
******
   Confidence          Name
0   99.984337       Apparel
1   99.984337      Clothing
2   99.215866           Hat
3   99.215866           Cap
4   99.215866  Baseball Cap
5   67.547684       Sun Hat
************
******
   Confidence  Name
0   85.833984  Rope
************
******
   Confidence      Name
0   99.937042  Weaponry
1   99.937042    Weapon
2   99.937042       Gun
3   98.770020   Handgun
************
******
   Confidence  Name
0   85.709328  Text
************
******
    Confidence          Name
0    93.992134  Baseball Cap
1    93.992134           Cap
2    93.992134      Clothing
3    93.992134       Apparel
4    93.992134           Hat
5    88.026985        Fossil
6    78.663017   Accessories
7    78.663017     Accessory
8    76.515053      Gemstone
9    76.515053       Jewelry
10   61.389351       Mineral
************
******
   Confidence       Name
0   83.864136      Mixer
1   83.864136  Appliance
2   76.862610      Light
3   62.323692       Lamp
***********

************
******
   Confidence        Name
0   99.987587     Apparel
1   99.987587    Clothing
2   86.668030      Person
3   86.668030       Human
4   86.124847       Shirt
5   82.609993     T-Shirt
6   70.677666  Undershirt
7   62.109196      Sleeve
************
******
   Confidence         Name
0   98.743423       Person
1   98.743423     Exercise
2   98.743423  Working Out
3   98.743423       Sports
4   98.743423        Sport
5   98.743423        Human
6   98.568970      Fitness
7   96.008553          Gym
8   87.774765      Sitting
************
******
   Confidence        Name
0   83.965256       Plant
1   69.801559    Outdoors
2   60.906281  Vegetation
************
******
   Confidence         Name
0   98.578407       Person
1   98.578407        Human
2   98.192810        Sport
3   98.192810       Sports
4   98.192810     Exercise
5   98.192810  Working Out
6   97.933441      Fitness
7   96.255875          Gym
************
******
   Confidence      Name
0   77.552597  Lighting
1

************
******
    Confidence        Name
0    98.860451        Word
1    90.919403        Text
2    89.423141    Alphabet
3    86.601608      Symbol
4    86.601608      Number
5    75.775421  Team Sport
6    75.775421      Sports
7    75.775421       Sport
8    75.775421  Volleyball
9    75.775421        Team
10   75.239601    Clothing
11   75.239601     Apparel
12   61.651688  Scoreboard
************
******
   Confidence           Name
0   85.645134  Digital Watch
1   77.711876        Machine
2   77.711876       Gas Pump
3   77.711876           Pump
4   74.658165     Wristwatch
************
************
******
   Confidence       Name
0   99.686768       Tire
1   82.748657  Car Wheel
2   82.748657    Machine
3   82.748657      Wheel
************
******
   Confidence       Name
0   99.562355      Human
1   99.562355     Person
2   68.809029  Furniture
3   68.809029   Tabletop
************
******
   Confidence    Name
0   72.531296   Human
1   72.531296  Person
************
******